In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [7]:
lin = nn.Linear(5, 3)

data = torch.randn(2, 5)

print(F.relu(data))

tensor([[0.0000, 0.3501, 0.0000, 0.0000, 0.6272],
        [1.0935, 0.0939, 1.2381, 0.0000, 0.5119]])


In [12]:
data = torch.randn(5)
print(F.softmax(data, dim=0))
print(F.log_softmax(data, dim=0))

tensor([0.1206, 0.0079, 0.1592, 0.0855, 0.6267])
tensor([-2.1149, -4.8401, -1.8375, -2.4593, -0.4672])


In [16]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)


In [20]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)
        
def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [30]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [37]:
with torch.no_grad():
    sample = data[0]
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    log_probs = model(bow_vector)

print(next(model.parameters())[:, word_to_ix["creo"]])

for epoch in range(100):
    for instance, label in data:
        # Step 1 - clear accumulates gradients
        model.zero_grad()
        
        # Step 2.Make Bow vector 
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)
        
        # Step 3. Run forward pass
        log_probs = model(bow_vec)
        
        # Step 4. Compute loss, gradients, and update parameters
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)
        
        
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([ 0.5354, -0.4746], grad_fn=<SelectBackward>)
tensor([[-0.0485, -3.0509]])
tensor([[-3.5480, -0.0292]])
tensor([ 0.5514, -0.4906], grad_fn=<SelectBackward>)
